# LOCA2 Inventory Checks

## Libraries

In [ ]:
######################################################
#
# Libraries
#


import os    as os

import numpy as np

import xarray as xr

import pandas as pd

import matplotlib.pyplot as plt

#
######################################################

## File Control

In [ ]:
######################################################
#
# File Control
#

working_directory  = "./"

thredds_root_dir   = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS/"


loca2_mod_member_file = "LOCA2_Model_Member_Complete_List.csv"




Scenarios = ["historical",
                 "ssp245",
                 "ssp370",
                 "ssp585"]

Vars = ["tasmax",
        "tasmin",
            "pr"]

#
######################################################

## Pull All Possible Models and Members

In [ ]:
######################################################
#
# Pull Full Inventory List
#

full_models_and_members_list = pd.read_csv(filepath_or_buffer = loca2_mod_member_file)

full_models_and_members_list

n_records = len(full_models_and_members_list)

display(full_models_and_members_list)

display(n_records)

#
######################################################

In [ ]:
loca2_mod_member_file

## Scan Disk for Available Files

In [ ]:
######################################################
#
# Scan Disk for Available Files
#

available_rows =  [False for i in range(len(full_models_and_members_list))]


for i in range(len(full_models_and_members_list)):
    
    for k in range(len(Scenarios)):
    
        file_name  = "LOCA2-CONUS___" + full_models_and_members_list.at[i,"Model"]+"."+ full_models_and_members_list.at[i,"Member"]+"___"+Scenarios[k]+".nc"

        #print(file_name)

        file_name  = thredds_root_dir + Scenarios[k] + "/" + file_name

        if os.path.isfile(file_name):

            my_string_all = "---"
            my_string_max = "-"
            my_string_min = "-"
            my_string_pcp = "-"
            xf         = xr.open_dataset(filename_or_obj = file_name)

            if "tasmax" in list(xf.keys()):
                my_string_max = "X"
                available_rows[i] = True
            if "tasmin" in list(xf.keys()):
                my_string_min = "N"        
                available_rows[i] = True
            if "pr"     in list(xf.keys()):
                my_string_pcp = "P"    
                available_rows[i] = True
            full_models_and_members_list.at[i,  Scenarios[k]] = my_string_max+my_string_min+my_string_pcp

            


full_models_and_members_list.sort_values(by           = ["Rank","NetError"],
                                         inplace      =                True,
                                         ignore_index =                True)
        
#
######################################################

## Save Data 

In [ ]:
######################################################
#
# Save Full Inventory
#

full_models_and_members_list.to_csv(path_or_buf = working_directory + loca2_mod_member_file,
                                   index        =  False,
                                   na_rep       =     '')

#
######################################################

# Cut only available files

In [ ]:
######################################################
#
# Trim Only Available Ones
#

avail_models_and_members_list = full_models_and_members_list.copy()

avail_models_and_members_list = avail_models_and_members_list[available_rows]

avail_models_and_members_list.sort_values(by           = ["Rank","NetError"],
                                          inplace      =                True,
                                          ignore_index =                True)
                                                              
                                                              
display(avail_models_and_members_list)

avail_models_and_members_list.to_csv(path_or_buf = working_directory + "LOCA2_Model_Member_Available_List.csv",
                                    index        =  False,
                                    na_rep       =     '')

os.system("scp ./LOCA2_Model_Member_Available_List.csv wjc@kyrill:"+thredds_root_dir)
os.system("scp ./LOCA2_Model_Member_Complete_List.csv wjc@kyrill:"+thredds_root_dir)

#
######################################################